In [5]:
# Code from Harish Satgunarajah
import pandas as pd

df = pd.read_csv("../FIBAData.csv", index_col = 0, encoding='latin-1')
df.head()

array(['2019 World Cup', '2019 World Cup African Qualifiers',
       '2019 World Cup American Qualifiers',
       '2019 World Cup Asian Qualifiers',
       '2019 World Cup Europe Qualifiers', '2017 AmeriCup',
       '2017 AsiaCup', '2017 EuroBasket', '2017 AfroBasket',
       '2016 Olympics', '2016 Olympic Qualifying Tournaments',
       '2015 AfroBasket', '2015 AmeriCup', '2015 EuroBasket',
       '2015 AsiaCup', '2014 World Cup', '2013 AmeriCup',
       '2013 EuroBasket', '2013 AsiaCup'], dtype=object)

In [ ]:
a
index = -1
previous = ""
def getGameID(row):
    global index
    global previous
    if previous != (row.Region + str(row.Date) + row["Away Team"] + row["Home Team"]):
        index += 1
        previous = (row.Region + str(row.Date) + row["Away Team"] + row["Home Team"])
    row["GameID"] = index
    return row
data = data.apply(getGameID, axis='columns')   

success_type = {"Make 2 Pts":2, "Make 3 Pts":3, "1 Pts":1, "Made":1}

def getPts(result):
    if result not in success_type:
        return 0
    return success_type[result]

def getPlayPts(play, playType):
    if (playType in play):
        shot = getShotType(play)
        return getPts(shot)
    return 0
def getPlayCount(playSeries, playType):
    checkPlay = lambda play : playType in play
    mapped = playSeries.map(checkPlay).value_counts()
    if True in mapped:
        return playSeries.map(checkPlay).value_counts()[True]
    return 0
def checkFailurePlay(play, playType):
    return playType in play and getShotType(play) not in success_type
def checkSuccessfulPlay(play, playType):
    return playType in play and getShotType(play) in success_type
def getShotType(play):
    index = play.rfind(">")
    return play[index+2:]
def getPlayEfficiency(playSeries, t):
    return playSeries.apply(getPlayPts, playType=t).sum()/getPlayCount(playSeries, t)
    
def getPossessions(dataset):
    series = dataset.loc[(dataset.Result.notnull()) & (dataset.Result != "Run Offense") & (dataset.Result != "Free Throw")]["Result"]
    return len(series)


def getPlayType(play):
    start = play.find(">")
    end = play.find(">", start+1)
    return play[start+2:end-1]

def getPlayTypeTwo(play):
    if "Ball Delivered" in play:
        ball_delivered = play.find("Ball Delivered >")
        first_arrow = play.find(">", ball_delivered)
        second_arrow = play.find(">", first_arrow + 1)
        third_arrow = play.find(">", second_arrow + 1)
        return play[second_arrow+2:third_arrow-1]
    else:
        return ""

def getStats(country, tournament=-1, ID=-1):
    d = dict()
    if (ID != -1):
        print(country, ID)
        dataset = data.loc[(data.GameID == ID) & (data.Team == country)];
    else:
        d["Offensive/Defensive"] = ["Offensive"]
        d["Country"] = [country]
        d["Tournament"] = [tournament]
        #print(country, tournament)
        dataset = data.loc[(data.Team == country) & (data.Region == tournament)];
        #print("Number of Games", len(dataset["GameID"].unique()));
        d["# Games"] = [len(dataset["GameID"].unique())];
    series = dataset.loc[(dataset.Result.notnull())]["Play Details"];
    playTypes = series.apply(getPlayType);
    secondPlayTypes=series.apply(getPlayTypeTwo);
    typeCounts = playTypes.value_counts();
    secondCounts = secondPlayTypes.value_counts();
    if "P&R Roll Man" in secondCounts:
        typeCounts["P&R Roll Man"] = 0;
    typeCounts.update(secondCounts);
    if (tournament==-1):
        lastRow = data.loc[data.GameID == ID].iloc[-1]
        if(country == lastRow["Home Team"]):
            print("Total Pts", lastRow["Home Score"])
        else:
            print("Total Pts", lastRow["Away Score"])
    
    #print("Number of Possessions", getPossessions(dataset))
    d["# Possessions"] = [getPossessions(dataset)];
    play_types = ['Hand Off', 'P&R Ball Handler', 'Cut', 'Offensive Rebound', 'Spot-Up',
 'No Play Type', 'Transition', 'Non Possession', 'Off Screen', 'Post-Up', 'ISO', "P&R Roll Man"]
    play_types = list((set(play_types)).union(set(playTypes)));
    play_types.remove('Non Possession')
    for play in play_types:
        if play in typeCounts:
            #print(play, round(getPlayEfficiency(series, play),3), typeCounts[play]);
            d[play + " Pts per Possession"] = 0
            d[play + " Pts per Possession"] = [round(getPlayEfficiency(series, play),3)];
            d[play + " Play %"] = 0
            d[play + " Play %"] = [round((typeCounts[play]/d["# Possessions"])[0]*100,2)];
    return pd.DataFrame(data=d)


            
            
def getDefensiveStats(country, tournament=-1):
    d = dict();
    d["Offensive/Defensive"] = ["Defensive"]
    d["Country"] = [country]
    d["Tournament"] = [tournament]
    dataset = data.loc[((data["Home Team"] == country) | (data["Away Team"] == country)) & (data.Region == tournament)];
    dataset = dataset.loc[data.Team != country];
    series = dataset.loc[(dataset.Result.notnull())]["Play Details"];
    playTypes = series.apply(getPlayType);
    secondPlayTypes=series.apply(getPlayTypeTwo);
    typeCounts = playTypes.value_counts();
    secondCounts = secondPlayTypes.value_counts();
    if "P&R Roll Man" in secondCounts:
        typeCounts["P&R Roll Man"] = 0;
    typeCounts.update(secondCounts);
    d["# Games"] = [len(dataset["GameID"].unique())];
    #print("Number of Possessions", getPossessions(dataset))
    d["# Possessions"] = [getPossessions(dataset)];
    play_types = ['Hand Off', 'P&R Ball Handler', 'Cut', 'Offensive Rebound', 'Spot-Up',
 'No Play Type', 'Transition', 'Non Possession', 'Off Screen', 'Post-Up', 'ISO', "P&R Roll Man"]
    play_types = list((set(play_types)).union(set(playTypes)));
    play_types.remove('Non Possession')
    for play in play_types:
        if play in typeCounts:
            #print(play, round(getPlayEfficiency(series, play),3), typeCounts[play]);            
            d[play + " Pts per Possession"] = 0
            d[play + " Pts per Possession"] = [round(getPlayEfficiency(series, play),3)];
            d[play + " Play %"] = 0
            d[play + " Play %"] = [round((typeCounts[play]/d["# Possessions"])[0]*100,2)];
    return pd.DataFrame(data=d)


newData = pd.DataFrame()    

teams = ['Spain', 'Argentina', 'Australia', 'France', 'Serbia', 'Czech Republic', 'USA', 'Poland', 'Brazil', 'Germany', 'Canada', 'New Zealand', 'Turkey', 'Greece', 'Lithuania', 'Dominican Republic', 'Senegal', 'Jordan', 'Montenegro', 'Japan', 'Nigeria', 'China', 'Iran', 'Philippines', 'Italy', 'Puerto Rico', 'Russia', 'Venezuela', 'Korea', 'Ivory Coast', 'Tunisia', 'Angola', 'Mali', 'Central African Republic', 'Rwanda', 'Morocco', 'Chad', 'Cameroon', 'Egypt', 'Guinea', 'Uganda', 'Democratic Republic of Congo', 'Mozambique', 'Uruguay', 'Mexico', 'Panama', 'Virgin Islands', 'Chile', 'Colombia', 'Bahamas', 'Paraguay', 'Cuba', 'Kazakhstan', 'Qatar', 'Lebanon', 'Syria', 'Chinese Taipei', 'Iraq', 'Hong Kong', 'India', 'Netherlands', 'Latvia', 'Ukraine', 'Slovenia', 'Hungary', 'Croatia', 'Bosnia and Herzegovina', 'Bulgaria', 'Estonia', 'Georgia', 'Israel', 'Finland', 'Belgium', 'Iceland', 'Austria', 'Great Britain', 'Kosovo', 'Belarus', 'Romania', 'Sweden', 'South Africa', 'Algeria', 'Gabon', 'Cape Verde', 'Zimbabwe', 'Macedonia', 'Palestine', 'Singapore', 'Kuwait', 'Malaysia', 'Jamaica', 'Bahrain', 'Saudi Arabia']

print("Starting stat generation")
for region in ['2019 World Cup' ,'2019 World Cup American Qualifiers',
 '2019 World Cup Asian Qualifiers', '2019 World Cup Europe Qualifiers',
 '2017 AmeriCup', '2017 AsiaCup', '2017 EuroBasket', '2016 Olympics',
 '2016 Olympic Qualifying Tournaments', '2015 AmeriCup', '2015 EuroBasket',
 '2015 AsiaCup', '2014 World Cup', '2013 AmeriCup', '2013 EuroBasket',
 '2013 AsiaCup']:
    for team in teams:
        print(region, team)
        temp = getStats(country=team, tournament=region)
        temp = temp.loc[temp["# Games"] != 0]
        newData = newData.append(temp, sort=True)
        temp = getDefensiveStats(country=team, tournament=region)
        temp = temp.loc[temp["# Games"] != 0]
        newData = newData.append(temp, sort=True)
        
newData.to_csv("Data/FullData.csv")